## Shitty language model

Here are the pieces to build and train a small shitty language model plus tokenizer from scratch, on Tom Lehrer lyrics, generating insightful extensions such as:

*There once was a man named Oedipus Rex.*

*You may have heard about his odd complex.*

*His name appears in Freud's index,*

*'Cause he loved **himkas bin, moden, sob**.*

References:
* fastai [nb 27](https://github.com/fastai/course22p2/blob/master/nbs/27_attention.ipynb)

Cross attention:
- https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html
- https://arxiv.org/abs/2112.10752 -> inserting class / text embedding into K and V of attention

ResBlock probably needs a cross-attention call that takes the label (K,V) and the processed image (Q) and returns something of the shape of the processed image

paper: U-Net: Convolutional Networks for Biomedical Image Segmentation

unet data: https://forum.image.sc/t/isbi-2012-site-down/57867

minbpe
* https://www.youtube.com/watch?v=zduSFxRajkE

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import random_neural_net_models.learner as rnnm_learner
import random_neural_net_models.text as rnnm_text
import random_neural_net_models.tokenization as rnnm_tok
import random_neural_net_models.transformer as rnnm_trans
import random_neural_net_models.utils as utils
from torch.utils.data import RandomSampler

In [ ]:
utils.make_deterministic(42)

In [ ]:
device = utils.get_device()
device

tom lehrer's songs: https://tomlehrersongs.com/

In [ ]:
path = Path("../data/tom-lehrer")

In [ ]:
files = rnnm_text.find_files(path, "*.txt")
files

In [ ]:
body_for_tokenizer = rnnm_text.concat_files(files, "\n")
body_for_tokenizer

In [ ]:
vocab_size = 200
tokenizer = rnnm_tok.TokenizerRegex()
tokenizer.fit(
    body_for_tokenizer,
    vocab_size=vocab_size,
    pattern=rnnm_tok.GPT4_SPLIT_PATTERN,
)

In [ ]:
special_token2id_map = {
    "<|endoftext|>": 100257,
    "<|fim_prefix|>": 100258,
    "<|fim_middle|>": 100259,
    "<|fim_suffix|>": 100260,
    "<|endofprompt|>": 100276,
}
tokenizer.register_special_tokens(special_token2id_map)

In [ ]:
block_size = 128
ds_train = rnnm_text.TextDataset(
    path=path,
    suffix="*.txt",
    tokenizer=tokenizer,
    block_size=block_size,
    end_of_text_token="<|endoftext|>",
)

In [ ]:
ds_train[0]

In [ ]:
ds_train.text_to_dense_ids(
    "Encoding some random string to ids for model inference"
)

In [ ]:
bs_train = 10
sampler = RandomSampler(
    ds_train,
    replacement=True,
    num_samples=int(1e5),
    generator=torch.manual_seed(3407),
)
dl_train = DataLoader(
    ds_train,
    batch_size=bs_train,
    sampler=sampler,
    collate_fn=rnnm_text.collate_text_dataset_to_block,
)

In [ ]:
next(iter(dl_train))

In [ ]:
num_blocks = 2
emb_dim = 10
n_tokens = block_size
latent_dim = 40
num_heads = 4

model = rnnm_trans.LanguageModelWithTensordict(
    vocab_size=ds_train.vocab_size,
    emb_dim=emb_dim,
    n_tokens=n_tokens,
    latent_dim=latent_dim,
    num_heads=num_heads,
    num_blocks=num_blocks,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_trans.CrossEntropyLoss()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=2, lr_find_callback=lr_find_callback
)

In [ ]:
lr_find_callback.plot(yscale="log")

In [ ]:
learning_rate = 3e-2
n_epochs = 5

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
# takes about 10min with a small gpu
learner.fit(dl_train, n_epochs=n_epochs)

In [ ]:
loss_callback.plot()

inference over some examples in the training dataset

In [ ]:
inp = next(iter(dl_train))
inp

In [ ]:
out_ids_dense = model.generate(inp.to(device), max_new_tokens=20)
out_ids_dense

In [ ]:
ds_train.dense_ids_to_strings(out_ids_dense.cpu())

inference over some user given text (needs to be at least of size `block_size`)

In [ ]:
user_text = """From the Bible to the popular song
There's one theme that we find right along.
Of all ideals they hail as good
The most sublime is Motherhood.
There was a man, though, who, it seems,
Once carried this ideal to extremes.
He loved his mother and she loved him,
And yet his story is rather grim"""

In [ ]:
user_token_ids = ds_train.text_to_dense_ids(user_text)
user_token_ids = rnnm_text.TokenIDBlockX(x=user_token_ids, batch_size=[1])
user_token_ids

In [ ]:
out_ids_dense = model.generate(user_token_ids.to(device), max_new_tokens=60)
out_ids_dense

In [ ]:
model_text = ds_train.dense_ids_to_strings(out_ids_dense.cpu())[0]
print("Input:")
print(user_text)
print("\nInference:")
print(model_text)

**Input:**

*From the Bible to the popular song*

*There's one theme that we find right along.*

*Of all ideals they hail as good*

*The most sublime is Motherhood.*

*There was a man, though, who, it seems,*

*Once carried this ideal to extremes.*

*He loved his mother and she loved him,*

*And yet his story is rather grim*

**Inference:**

*ugh, who, it seems,*

*Once carried this ideal to extremes.*

*He loved his mother and she loved him,*

*And yet his story is rather grim*<u>:


There onle

S nd taved, beeich sisis sond ise th ad jouks</u>

beeich sisis sond ise th ad jouks indeed.